<a href="https://colab.research.google.com/github/kiruthikpurpose/SimpleLMCohereAPI/blob/main/SimpleLanguageModelUsingCohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cohere #Manually install Cohere if not working
import numpy as np
import warnings
from annoy import AnnoyIndex
import pandas as pd

warnings.filterwarnings('ignore')


In [ ]:
# Define the question
question = "Which Indian beach should I visit?"

# Define the text
text = """
India is renowned for its diverse and beautiful beaches, each offering unique experiences and attractions. From bustling, vibrant beaches to serene, secluded ones, there's something for everyone.

Goa Beaches: Goa is famous for its numerous beaches, each with its own charm. Baga Beach is known for its lively nightlife, water sports, and vibrant atmosphere. Palolem Beach offers a more tranquil experience with its scenic beauty and quieter environment.

Marina Beach: Located in Chennai, Marina Beach is one of the longest urban beaches in the world. It's popular for its golden sands, bustling local life, and the nearby attractions like historical monuments and food stalls.

Varkala Beach: Situated in Kerala, Varkala Beach is known for its stunning cliffs and natural springs. It's a great place for relaxation, swimming, and enjoying the scenic views. The beach is also known for its Ayurvedic treatments and yoga centers.

Radhanagar Beach: Located on Havelock Island in the Andaman and Nicobar Islands, Radhanagar Beach is famous for its pristine white sands and clear blue waters. It's an ideal spot for swimming, snorkeling, and enjoying the natural beauty.

Kovalam Beach: Also in Kerala, Kovalam Beach is famous for its crescent-shaped coastline, lighthouses, and calm waters. It's a popular destination for tourists looking for a mix of relaxation and adventure.

Puri Beach: Located in Odisha, Puri Beach is known for its cultural significance and beautiful golden sands. It's a popular spot for both pilgrims and tourists, offering a unique blend of spirituality and scenic beauty.
"""


In [ ]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts if t])


In [ ]:
# Initialize Cohere client
co = cohere.Client('<Get your Cohere API Key from cohere.com>')

# Get the embeddings
response = co.embed(texts=texts.tolist()).embeddings


In [ ]:
# Check the dimensions of the embeddings
embeds = np.array(response)

# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')


In [ ]:
def search_text(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings

    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0], 10, include_distances=True)

    search_results = texts[similar_item_ids[0]]

    return search_results

# Example query
results = search_text(question)
print(results[0])


In [ ]:
def ask_llm(question, num_generations=1):
    # Search the text archive
    results = search_text(question)

    # Get the top result
    context = results[0]

    # Prepare the prompt
    prompt = f"""
    More information about Indian beaches:
    {context}
    Question: {question}

    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,
    reply that the answer is not available.
    """

    prediction = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations


In [ ]:
# Example question 1
question = "Which Indian beach should I visit?"
results = ask_llm(question)
print(results[0])

# Example question 2
question = "Which Indian beach is family-friendly?"
results = ask_llm(question)
print(results[0])

# Example question 3
question = "Which Indian beach has a lighthouse?"
results = ask_llm(question)
print(results[0])
